In [1]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:

import torch
from torch import *
dataset_x = torch.load("/kaggle/input/zip-40k-dataset-cxr/final_x/final_x.pt")
dataset_y = torch.load("/kaggle/input/zip-40k-dataset-cxr/final_y (1).pt")

In [3]:
from sklearn.model_selection import train_test_split
xtr, xts, ytr, yts = train_test_split(dataset_x, dataset_y, test_size=0.2, shuffle=True, stratify=dataset_y)

In [4]:
xtr.size()

torch.Size([32301, 1, 224, 224])

In [5]:
xts.size()

torch.Size([8076, 1, 224, 224])

In [6]:
import torch
from torch import *
my_dataset = torch.utils.data.TensorDataset(xtr,ytr) # create your datset
my_dataloader = torch.utils.data.DataLoader(my_dataset,batch_size=32,
                                          shuffle=True, num_workers=2)

my_dataset = torch.utils.data.TensorDataset(xts,yts) # create your datset
my_tdataloader = torch.utils.data.DataLoader(my_dataset,batch_size=32)

In [7]:
len(my_dataloader)

1010

In [8]:
from IPython.display import clear_output

In [9]:
rep = []

# model

In [14]:
import os
import time
from tempfile import TemporaryDirectory

import cv2
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt

import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, random_split

import torch
import torchvision.models as models

device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 



import timm

# from timm
pretrained_model_name = "inception_v3"
model = timm.create_model(pretrained_model_name, pretrained=True)
print(model)
num_channels = 1  # for grayscale images, but it could be any number
# Extract the first conv layer's parameters
num_filters = model.Conv2d_1a_3x3.conv.out_channels
kernel_size = model.Conv2d_1a_3x3.conv.kernel_size
stride = model.Conv2d_1a_3x3.conv.stride
padding = model.Conv2d_1a_3x3.conv.padding
conv1 = torch.nn.Conv2d(num_channels, num_filters, kernel_size=kernel_size, stride=stride, padding=padding)
original_weights = model.Conv2d_1a_3x3.conv.weight.data.mean(dim=1, keepdim=True)
conv1.weight.data = original_weights.repeat(1, num_channels, 1, 1)
model.Conv2d_1a_3x3.conv = conv1



# Freeze only the convolutional layers of the pre-trained model
for param in model.parameters():
    if isinstance(param, nn.Conv2d):
        param.requires_grad = False



# Modify the model head for fine-tuning
num_features = 2048

# Additional linear layer and dropout layer
model.fc = nn.Sequential(
    nn.Linear(num_features, 256),  # Additional linear layer with 256 output features
    nn.ReLU(inplace=True),         # Activation function (you can choose other activation functions too)
    nn.Dropout(0.5),               # Dropout layer with 50% probability
    nn.Linear(256, 4),    # Final prediction fc layer
    nn.Softmax()
)

model = torch.nn.DataParallel(model, device_ids = [0,1]).to(device)

criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)



batch_size=32
num_epochs=30
val_size = len(my_tdataloader)
train_size = len(my_dataloader) 

losses = [] 
accuracies = [] 
val_losses = [] 
val_accuracies = [] 
# Train the model 
cnt = 0
x = range(0,num_epochs)
for epoch in x: 
    cnt = 0
    for i, (images, labels) in enumerate(my_dataloader): 
        # Forward pass 
        images=images.to(device) 
        labels=labels.type(torch.LongTensor).to(device) 
        outputs = model(images) 
        loss = criterion(outputs, labels) 
        
        # Backward pass and optimization 
        optimizer_ft.zero_grad() 
        loss.backward() 
        optimizer_ft.step() 
        cnt = cnt+1
        print(cnt)
        clear_output()
        _, predicted = torch.max(outputs.data, 1)
        
    acc = (predicted == labels).sum().item() / labels.size(0) 
    accuracies.append(acc) 
    losses.append(loss.item()) 

    # Evaluate the model on the validation set	
    rep.append('Epoch [{}/{}],Loss:{:.4f},Accuracy:{:.2f}'.format( 
        epoch+1, num_epochs, loss.item(),acc))
    print('Epoch [{}/{}],Loss:{:.4f},Accuracy:{:.2f}'.format( 
        epoch+1, num_epochs, loss.item(),acc))







KeyboardInterrupt: 

In [ ]:
rep

# Evaluation

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sn
import pandas as pd

y_pred = []
y_true = []  

# iterate over test data
for inputs, labels in my_tdataloader:
        output = model(inputs) # Feed Network

        output = (torch.max(torch.exp(output), 1)[1]).data.cpu().numpy()
        y_pred.extend(output) # Save Prediction
        
        labels = labels.data.cpu().numpy()
        y_true.extend(labels) # Save Truth

# constant for classes
classes = ('COVID','Normal','Pneumonia','Bacterial infection')

# Build confusion matrix
cf_matrix = confusion_matrix(y_true, y_pred)
df_cm = pd.DataFrame(cf_matrix / np.sum(cf_matrix, axis=1)[:, None], index = [i for i in classes],
                     columns = [i for i in classes])
plt.figure(figsize = (12,7))
sn.heatmap(df_cm, annot=True)
plt.savefig('output.png')


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_true, y_pred))
classification_report(y_true, y_pred)